In [1]:
import pandas as pd
import my_utils as mu
import json
import logging
import requests as req
import time

In [8]:
def rawdataLoader(datas: str):
    return json.loads(datas.replace("'", '"').replace('True', '1').replace('False', '0'))

### eventExtractor() 수정 사항

In [ ]:
public = [
    'OBJECTIVE_BOUNTY_PRESTART',
    'GAME_END',
    'PAUSE_END'
]
private = [
    ('CHAMPION_KILL', 'killerId', int),
    ('BUILDING_KILL', 'killerId', int),
    ('TURRET_PLATE_DESTROYED', 'killerId', int),
    ('ELITE_MONSTER_KILL', 'killerId', int),
    ('CHAMPION_SPECIAL_KILL', 'killerId', int),
    ('WARD_KILL', 'killerId', int),
    ('WARD_PLACED', 'creatorId', int)
]
adder = [
    ('CHAMPION_ASSIST', 'assistingParticipantIds', list),
    ('CHAMPION_DEATH', 'victimId', int)
]